In [103]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os
from pathlib import Path
import numpy as np

In [104]:
folder = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/GDP")
for f in folder.iterdir():
    print(f.name)

GDP per CAPITA - Maddison Project .xlsx
mpd2023_web.xlsx


In [105]:
Project_Countries = [
    'Australia','Canada','Denmark','Finland','France','Germany',
    'Italy','Japan','Netherlands','Norway','Portugal','Spain',
    'Sweden','Switzerland','United Kingdom','United States'
]

DATA EXTRACTION MAKING THE VARIABLE ---> REAL GDP per CAPITA 

In [106]:
# In the Paper they use the MAddison Project Database, For Japan missing data (1882, 1884)
# they use the 1885 value of real GDP per capita and population figures for 1882–1884 
# to back-calculate real GDP. 
# Taking constant real GDP in 1885, we divide it by the annual population in 1882, 1883 and 1884 
# to estimate real GDP per capita for those three years. This method provides a reasonable proxy 
# in 2011 international dollars. At the end of the day, we end up with data for the full sample of 16 countries 
# from 1882 to 2020.

# DATA file the Maddison Project Database found in the GDP folder in the DATA folder.

path_maddison_project = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/GDP/GDP per CAPITA - Maddison Project .xlsx")

df_maddison_project = pd.read_excel(path_maddison_project, sheet_name='Full data')

df_OECD_GDP = df_maddison_project[df_maddison_project['country'].isin(Project_Countries)]
df_OECD_GDP = df_OECD_GDP[['country', 'year', 'gdppc', 'pop']]


path_maddison_project_2022 = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/GDP")
df_maddison_project_2022 = pd.read_excel(path_maddison_project_2022 / "mpd2023_web.xlsx", sheet_name= 'GDPpc')
df_maddison_project_2022 = df_maddison_project_2022[['Australia','Canada','Denmark','Finland','France','Germany','Italy','Japan','Netherlands','Norway','Portugal','Spain','Sweden','Switzerland','United Kingdom','United States', 'GDP pc 2011 prices']] # weird table have to get data like this 
df_maddison_project_2022 = df_maddison_project_2022.rename(columns={'GDP pc 2011 prices': 'year'})
df_maddison_project_2022['year'] = pd.to_numeric(df_maddison_project_2022['year'], errors='coerce')
df_maddison_project_2022 = df_maddison_project_2022.dropna(subset=['year']).astype({'year': 'int'})
df_maddison_project_2022_years = df_maddison_project_2022[df_maddison_project_2022["year"].isin([2018, 2019, 2020])]
df_maddison_project_2022_melted = df_maddison_project_2022_years.melt(id_vars=['year'], var_name='country', value_name='gdppc_2022')


years_to_add = [2019, 2020]

# long Maddison data for 2019–2020
new_rows = df_maddison_project_2022_melted[df_maddison_project_2022_melted['year'].isin(years_to_add)].copy()
new_rows['pop'] = np.nan
new_rows = new_rows.rename(columns={'gdppc_2022': 'gdppc'})
new_rows = new_rows[['country', 'year', 'gdppc', 'pop']]
df_OECD_GDP = pd.concat([df_OECD_GDP, new_rows], ignore_index=True)
df_OECD_GDP = df_OECD_GDP.sort_values(['country', 'year']).reset_index(drop=True)

df_OECD_GDP = pd.merge(df_OECD_GDP,df_maddison_project_2022_melted,on=['country', 'year'], how='left')

# overwrite old gdppc wherever you have a value from the 2022 Maddison data
mask = df_OECD_GDP['gdppc_2022'].notna()
df_OECD_GDP.loc[mask, 'gdppc'] = df_OECD_GDP.loc[mask, 'gdppc_2022']
df_OECD_GDP = df_OECD_GDP.drop(columns=['gdppc_2022'])

df_OECD_GDP = df_OECD_GDP[(df_OECD_GDP['year'] >= 1882) & (df_OECD_GDP['year'] <= 2020)]




# Sorting out the data for Japan for the years 1882, 1883, 1884
mask_japan = df_OECD_GDP['country'] == 'Japan'
jp = df_OECD_GDP[mask_japan]

rgdp_1885 = (jp.loc[jp['year'] == 1885, 'gdppc'].iloc[0]* jp.loc[jp['year'] == 1885, 'pop'].iloc[0])
years_to_fill = [1882, 1883, 1884]
mask_japan_early = mask_japan & df_OECD_GDP['year'].isin(years_to_fill)

df_OECD_GDP.loc[mask_japan_early, 'gdppc'] = (rgdp_1885 / df_OECD_GDP.loc[mask_japan_early, 'pop'])

#df_OECD_GDP.head()



Labour Productivity 

In [107]:
Project_Countries = [
    'Australia','Canada','Denmark','Finland','France','Germany',
    'Italy','Japan','Netherlands','Norway','Portugal','Spain',
    'Sweden','Switzerland','United Kingdom','United States'
]

path_labour_productivity = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/Labour Productivity/Long Term Labour Productivity.xlsx")
df_labour_productivity = pd.read_excel(path_labour_productivity, sheet_name='Labor Productivity')

df_labour_productivity = df_labour_productivity[
    ['Year', 'AUS', 'AUT', 'BEL', 'CAN', 'DNK', 'FIN', 'FRA', 'DEU',
     'GRC', 'IRL', 'ITA', 'JPN', 'NLD', 'NZL', 'NOR', 'PRT', 'ESP',
     'SWE', 'CHE', 'GBR', 'USA']
]

replace_name_dict = {
    'AUS': 'Australia',
    'AUT': 'Austria',
    'BEL': 'Belgium',
    'CAN': 'Canada',
    'DNK': 'Denmark',
    'FIN': 'Finland',
    'FRA': 'France',
    'DEU': 'Germany',
    'GRC': 'Greece',
    'IRL': 'Ireland',
    'ITA': 'Italy',
    'JPN': 'Japan',
    'NLD': 'Netherlands',
    'NZL': 'New Zealand',
    'NOR': 'Norway',
    'PRT': 'Portugal',
    'ESP': 'Spain',
    'SWE': 'Sweden',
    'CHE': 'Switzerland',
    'GBR': 'United Kingdom',
    'USA': 'United States'
}

df_labour_productivity = df_labour_productivity.rename(columns=replace_name_dict)

df_labour_productivity = df_labour_productivity[
    (df_labour_productivity['Year'] >= 1890) &
    (df_labour_productivity['Year'] <= 2020)
]

columns_keep_df_labour_productivity = ["Year"] + Project_Countries
df_labour_productivity = df_labour_productivity[columns_keep_df_labour_productivity]
df_labour_productivity.head()

,Year,Australia,Canada,Denmark,Finland,France,Germany,Italy,Japan,Netherlands,Norway,Portugal,Spain,Sweden,Switzerland,United Kingdom,United States
90,1890,11.275570,4.185372,3.550943,2.023169,2.792736,4.814139,2.018100,1.020471,5.493740,3.641319,1.925517,4.093740,2.689252,5.961463,5.974749,4.655917
91,1891,11.346384,3.856720,3.622624,2.013744,2.798662,4.832038,2.087370,0.970538,5.331186,3.699415,1.899728,4.170664,2.619799,5.536116,5.999465,4.703721
92,1892,9.399105,3.786976,3.705531,1.930385,2.841674,4.893941,2.119804,1.034053,5.364090,3.734071,1.892896,4.459632,2.826159,5.927149,5.962112,4.626473
93,1893,8.518302,3.718733,3.764063,1.999724,2.756078,5.231450,2.181473,1.031995,5.351097,3.797245,1.935931,4.282428,2.976881,6.074320,5.974519,5.022901
94,1894,8.448742,3.847008,3.809196,2.107358,2.992517,5.341485,2.184280,1.154029,5.415974,3.785117,1.918264,4.309023,2.973899,5.848441,6.268525,5.131884


Consumer Price Index 

In [108]:
# Missibg Data from Ireland --> paper claims full data set but data set missing 1882 to 1921, linear interpolation not possible since we dont have a stating points cannot
# draw a trend line. 
path_CPI = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/CPI")
df_CPI_Jorda = pd.read_excel(path_CPI / " Jordá et al 2017  - .xlsx", sheet_name='Sheet1')
df_CPI_Jorda = df_CPI_Jorda[['country', 'year', 'cpi']]
df_CPI_Jorda = df_CPI_Jorda[(df_CPI_Jorda['year'] >= 1882) & (df_CPI_Jorda['year'] <= 2020)]

country_replace = {
    "AUS": "Australia",
    "AUT": "Austria",
    "BEL": "Belgium",
    "CAN": "Canada",
    "DNK": "Denmark",
    "FIN": "Finland",
    "FRA": "France",
    "DEU": "Germany",
    "GRC": "Greece",
    "IRL": "Ireland",
    "ITA": "Italy",
    "JPN": "Japan",
    "NLD": "Netherlands",
    "NZL": "New Zealand",
    "NOR": "Norway",
    "PRT": "Portugal",
    "ESP": "Spain",
    "SWE": "Sweden",
    "CHE": "Switzerland",
    "GBR": "United Kingdom",  
    "UK": "United Kingdom",    
    "USA": "United States",
    "US": "United States"      
}

df_CPI_Jorda["country"] = df_CPI_Jorda["country"].replace(country_replace)

df_CPI_Jorda = df_CPI_Jorda[df_CPI_Jorda['country'].isin(Project_Countries)]



Trade Openness --> Exports - Imports / GDP 

In [109]:
# Trade Openness index --> Exports - Imports / GDP, keeping in line with the paper methodology
#Trade data are primarily sourced from the Jord`a-Schularick-Taylor Macrohistory Database (Jord`a et al., 2017)
#For specific country-year gaps, we first supplement with imports and exports data from Federico and Tena-Junguito (2016)
#Aus- tralia (1914), Switzerland (1882–1884), and Germany (1914–1923)
# In cases where the Jord`a-Schularick-Taylor Macrohistory Database provides data on imports and exports but lacks nominal GDP figures, 
# we supplement with nominal GDP data from the Global Macro Database (Mu ̈ller et al., 2025)
# we estimate trade openness by linearly interpolating the ratio of trade to GDP.
# full sample of 16 countries from 1882 to 2020.

path_TradeOpenness_Jorda_data_set = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/Trade Openness")
df_TradeOpenness_Jorda_data_set = pd.read_excel(path_CPI / " Jordá et al 2017  - .xlsx", sheet_name='Sheet1')
df_TradeOpenness_Jorda_data_set = df_TradeOpenness_Jorda_data_set[['country', 'year', 'exports', 'imports','gdp']]

country_replace = {
    "AUS": "Australia",
    "AUT": "Austria",
    "BEL": "Belgium",
    "CAN": "Canada",
    "DNK": "Denmark",
    "FIN": "Finland",
    "FRA": "France",
    "DEU": "Germany",
    "GRC": "Greece",
    "IRL": "Ireland",
    "ITA": "Italy",
    "JPN": "Japan",
    "NLD": "Netherlands",
    "NZL": "New Zealand",
    "NOR": "Norway",
    "PRT": "Portugal",
    "ESP": "Spain",
    "SWE": "Sweden",
    "CHE": "Switzerland",
    "GBR": "United Kingdom",  
    "UK": "United Kingdom",    
    "USA": "United States",
    "US": "United States"      
}

df_TradeOpenness_Jorda_data_set["country"] = df_TradeOpenness_Jorda_data_set["country"].replace(country_replace)


df_TradeOpenness_Jorda_data_set = df_TradeOpenness_Jorda_data_set[df_TradeOpenness_Jorda_data_set["country"].isin(Project_Countries)]

df_TradeOpenness_GMD_nominal_GDP_path = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/Trade Openness")
df_TradeOpenness_GMD_nominal_GDP = pd.read_excel(df_TradeOpenness_GMD_nominal_GDP_path / "GMD.xlsx", sheet_name='data_final')
df_TradeOpenness_GMD_nominal_GDP = df_TradeOpenness_GMD_nominal_GDP[['countryname', 'year', 'nGDP', 'exports', 'imports']]
df_TradeOpenness_GMD_nominal_GDP = df_TradeOpenness_GMD_nominal_GDP.rename(columns={'countryname': 'country'})
df_TradeOpenness_GMD_nominal_GDP = df_TradeOpenness_GMD_nominal_GDP[df_TradeOpenness_GMD_nominal_GDP["country"].isin(Project_Countries)]

df_TradeOpenness = pd.merge(df_TradeOpenness_Jorda_data_set, df_TradeOpenness_GMD_nominal_GDP, on=['country', 'year'], how='left')

df_Spdf_Supplementary_STATS_Australia_1914_path = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/Trade Openness/Supplementary Trade STATS - AUSTRALIA (1914)")
df_Supplementary_STATS_Australia_1914 = pd.read_excel( df_Spdf_Supplementary_STATS_Australia_1914_path / "oceania_1826_1938_FTWTHD_201710_v01.xlsx", sheet_name='Current prices, 1913 borders')





Inputting the Missing Values for The exports and Imports and also the Nominal GDP 

In [110]:
# Australia (1914)
df_TradeOpenness.loc[
    (df_TradeOpenness["country"] == "Australia") &
    (df_TradeOpenness["year"] == 1914),
    "imports"
] = 343.33

df_TradeOpenness.loc[
    (df_TradeOpenness["country"] == "Australia") &
    (df_TradeOpenness["year"] == 1914),
    "exports"
] = 317.36


# Switzerland (1882, 1883, 1884)
swiss_years = [1882, 1883, 1884]

mask_ch = (
    (df_TradeOpenness["country"] == "Switzerland") &
    (df_TradeOpenness["year"].isin(swiss_years))
)

df_TradeOpenness.loc[mask_ch, "imports"] = [78.39, 87.47, 85.46]
df_TradeOpenness.loc[mask_ch, "exports"] = [134.97, 136.12, 136.68]


# Germany (1914–1923)
germany_years = [1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923]

mask_de = (
    (df_TradeOpenness["country"] == "Germany") &
    (df_TradeOpenness["year"].isin(germany_years))
)

df_TradeOpenness.loc[mask_de, "imports"] = [
    2023.86, 1690.52, 2000.05, 1690.52, 1690.52, 1858.55, 2051.43, 2112.12, 2122.92, 1529.60, 
]
df_TradeOpenness.loc[mask_de, "exports"] = [ 1761.95, 738.11, 904.78, 833.35, 1119.07, 1393.73, 1711.00, 1047.51, 1839.12, 1599.10 ] 

df_TradeOpenness = df_TradeOpenness[(df_TradeOpenness['year'] >= 1882) & (df_TradeOpenness['year'] <= 2020)]


In [111]:
def compute_trade_openness():
    global df_TradeOpenness   # <-- use your actual dataframe

    # 1. Primary measure: using exports_x and imports_x
    df_TradeOpenness['trade_openness_primary'] = (
        (df_TradeOpenness['exports_x'] + df_TradeOpenness['imports_x']) 
        / df_TradeOpenness['nGDP']
    )

    # 2. Define conditions for when the value is "weird" or invalid
    weird = (
        df_TradeOpenness['trade_openness_primary'].isna() |
        (df_TradeOpenness['trade_openness_primary'] < 0) |
        (df_TradeOpenness['trade_openness_primary'] > 3)
    )

    # 3. Fallback using exports_y and imports_y
    df_TradeOpenness.loc[weird, 'trade_openness'] = (
        (df_TradeOpenness.loc[weird, 'exports_y'] + df_TradeOpenness.loc[weird, 'imports_y'])
        / df_TradeOpenness.loc[weird, 'nGDP']
    )

    # 4. Keep primary values where valid
    df_TradeOpenness.loc[~weird, 'trade_openness'] = (
        df_TradeOpenness.loc[~weird, 'trade_openness_primary']
    )

    # Cleanup
    df_TradeOpenness.drop(columns=['trade_openness_primary'], inplace=True)

    return df_TradeOpenness

df_TradeOpenness = compute_trade_openness()

def interpolate_openness_for_country_years(df, country, start_year, end_year):
    mask_country = df["country"] == country
    df_country = df.loc[mask_country].sort_values("year").copy()
    df_country["openness_interp"] = df_country["trade_openness"].interpolate(method="linear")
    mask_gap_years = (df_country["year"] >= start_year) & (df_country["year"] <= end_year)
    df.loc[mask_country & mask_gap_years, "trade_openness"] = df_country.loc[mask_gap_years, "openness_interp"].values
    return df
# Germany: 1944–1947
df_TradeOpenness = interpolate_openness_for_country_years(df_TradeOpenness, "Germany", 1944, 1947)
# Japan: 1944–1945
df_TradeOpenness = interpolate_openness_for_country_years(df_TradeOpenness, "Japan", 1944, 1945)
# Netherlands: 1944–1945
df_TradeOpenness = interpolate_openness_for_country_years(df_TradeOpenness, "Netherlands", 1944, 1945)
# Germany 1914-1920
df_TradeOpenness = interpolate_openness_for_country_years(df_TradeOpenness, "Germany", 1914, 1923)
# Switzerland 1882-1884
df_TradeOpenness = interpolate_openness_for_country_years(df_TradeOpenness, "Switzerland", 1882, 1884)
# Australia 1914
df_TradeOpenness = interpolate_openness_for_country_years(df_TradeOpenness, "Australia", 1914, 1914)



df_TradeOpenness.describe()

,year,exports_x,imports_x,gdp,nGDP,exports_y,imports_y,imports,exports,trade_openness
count,2224.000000,2.202000e+03,2.202000e+03,2.204000e+03,2.223000e+03,2.209000e+03,2.209000e+03,14.000000,14.000000,2.224000e+03
mean,1951.000000,7.722444e+05,7.984612e+05,2.723247e+06,1.019707e+07,1.380302e+06,1.298392e+06,1383.195714,976.632143,4.052065e-01
std,40.133829,5.240291e+06,5.242390e+06,1.760694e+07,6.605239e+07,8.896349e+06,8.533765e+06,831.755417,627.832361,6.466756e-01
min,1882.000000,2.856000e-12,2.873000e-12,1.707798e-11,9.669400e-09,1.823821e-09,2.362936e-09,78.390000,134.970000,1.282452e-08
25%,1916.000000,5.325250e+00,5.376575e+00,5.126680e+01,1.477916e+03,3.203133e+02,3.345548e+02,639.897500,422.547500,4.547730e-04
50%,1951.000000,4.129968e+02,4.537720e+02,1.794046e+03,1.960228e+04,3.419691e+03,3.201383e+03,1690.520000,976.145000,1.717040e-01
75%,1986.000000,7.359207e+03,9.192000e+03,4.556449e+04,4.431389e+05,1.128150e+05,1.126349e+05,2017.907500,1547.757500,4.825879e-01
max,2020.000000,7.237242e+07,6.630599e+07,2.070466e+08,5.579109e+08,1.019464e+08,1.038285e+08,2122.920000,1839.120000,2.996546e+00


Liquid Liabilities Data Set 

In [112]:
# Liquid Liabilities Data Set comes from Jorda-Schularick-Taylor Macrohistory Database (Jorda et al., 2017)
# computed as M2 or m3 / nominal Gdp 
# e M2 estimates from Albert Ritschl’s dataset on interwar Germany (Ritschl, 2014)
#For Spain (1936–1939), we use M1 estimates from Mart ́ın-Acen ̃a (2018) (Table 1, page 34)
# For France (1914–1919), we rely on Bordo and Hautcoeur (2003) estimates of money growth, applying their reported growth rates 
# (30%, 70%, 27%, 28%, 40%, and 26%) to Jorda-Schularick-Taylor 1910 money stock level. Missing data for Japan (1945), 
# the Netherlands (1942–1944), Spain (1940), Germany (1923–1924, 1945–1947) have been linearly interpolated. 

path_Liquid_Liabilities = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/Liquid Liabilities")
df_Liquid_Liabilities = pd.read_excel(path_Liquid_Liabilities / "Jorda.xlsx", sheet_name="Sheet1")
df_Liquid_Liabilities = df_Liquid_Liabilities[["country", "year", "money", "gdp"]]
country_replace = {
    "AUS": "Australia",
    "AUT": "Austria",
    "BEL": "Belgium",
    "CAN": "Canada",
    "DNK": "Denmark",
    "FIN": "Finland",
    "FRA": "France",
    "DEU": "Germany",
    "GRC": "Greece",
    "IRL": "Ireland",
    "ITA": "Italy",
    "JPN": "Japan",
    "NLD": "Netherlands",
    "NZL": "New Zealand",
    "NOR": "Norway",
    "PRT": "Portugal",
    "ESP": "Spain",
    "SWE": "Sweden",
    "CHE": "Switzerland",
    "GBR": "United Kingdom",  
    "UK": "United Kingdom",    
    "USA": "United States",
    "US": "United States"      
}

df_Liquid_Liabilities["country"] = df_Liquid_Liabilities["country"].replace(country_replace)
df_Liquid_Liabilities = df_Liquid_Liabilities[df_Liquid_Liabilities["country"].isin(Project_Countries)]

# Germany (1939–1945)

ger_money_ritschl = {
    1939:  6.3278,   
    1940:  8.2882,   
    1941:  9.9844,
    1942:  12.9153,
    1943:  16.4405,
    1944: 21.0798,
}

for year, value in ger_money_ritschl.items():
    df_Liquid_Liabilities.loc[
        (df_Liquid_Liabilities["country"] == "Germany") &
        (df_Liquid_Liabilities["year"] == year),
        "money"
    ] = value


# Spain (1936–1939)

spain_money_martin_acena = {
    1936:  11461,
    1937:  22219,
    1938:  35852,
    1939:  42900,
}
for year, value in spain_money_martin_acena.items():
    df_Liquid_Liabilities.loc[(df_Liquid_Liabilities["country"] == "Spain") & (df_Liquid_Liabilities["year"] == year),"money"] = value


# France (1914–1919)

French_Growth_Rates_Bordo_Hautcoeur = {
    1914: 0.30,
    1915: 0.70,
    1916: 0.27,
    1917: 0.28,
    1918: 0.40,
    1919: 0.26,
}
M_1910 = df_Liquid_Liabilities.loc[(df_Liquid_Liabilities["country"] == "France") & (df_Liquid_Liabilities["year"] == 1910),"money"].iloc[0]
current_level = M_1910
for year in sorted(French_Growth_Rates_Bordo_Hautcoeur.keys()):
    g = French_Growth_Rates_Bordo_Hautcoeur[year]
    current_level = current_level * (1 + g)   
    df_Liquid_Liabilities.loc[
        (df_Liquid_Liabilities["country"] == "France") &
        (df_Liquid_Liabilities["year"] == year),
        "money"
    ] = current_level

df_Liquid_Liabilities["liquid_liabilities"] = (
    df_Liquid_Liabilities["money"] / df_Liquid_Liabilities["gdp"]
)

def interpolate_liq_for_country_years(df, country, start_year, end_year):
    mask_country = df["country"] == country
    df_country = df.loc[mask_country].sort_values("year").copy()
    df_country["liq_interp"] = df_country["liquid_liabilities"].interpolate(method="linear")
    mask_gap = (df_country["year"] >= start_year) & (df_country["year"] <= end_year)
    df.loc[mask_country & mask_gap, "liquid_liabilities"] = df_country.loc[mask_gap, "liq_interp"].values
    return df

# Japan (1945)
df_Liquid_Liabilities = interpolate_liq_for_country_years(df_Liquid_Liabilities, "Japan", 1945, 1945)

# Netherlands (1942–1944)
df_Liquid_Liabilities = interpolate_liq_for_country_years(df_Liquid_Liabilities, "Netherlands", 1942, 1944)

# Spain (1940)
df_Liquid_Liabilities = interpolate_liq_for_country_years(df_Liquid_Liabilities, "Spain", 1940, 1940)

# Germany (1923–1924)
df_Liquid_Liabilities = interpolate_liq_for_country_years(df_Liquid_Liabilities, "Germany", 1923, 1924)

# Germany (1945–1947)
df_Liquid_Liabilities = interpolate_liq_for_country_years(df_Liquid_Liabilities, "Germany", 1945, 1947)

# Wasn't mentioned in the paper but Norway has missing data for 1940 to 1945, applying same logic of linear interpolation
df_Liquid_Liabilities = interpolate_liq_for_country_years(df_Liquid_Liabilities, "Norway", 1940, 1945)

# Same for the Nethertheralns 1914 1921
df_Liquid_Liabilities = interpolate_liq_for_country_years(df_Liquid_Liabilities, "Netherlands", 1914, 1921)
df_Liquid_Liabilities = interpolate_liq_for_country_years(df_Liquid_Liabilities, "Netherlands", 1940, 1941)



df_Liquid_Liabilities = df_Liquid_Liabilities[(df_Liquid_Liabilities['year'] >= 1882) & (df_Liquid_Liabilities['year'] <= 2020)]

df_Liquid_Liabilities.describe()



,year,money,gdp,liquid_liabilities
count,2224.000000,2.215000e+03,2.204000e+03,2224.000000
mean,1951.000000,2.524930e+06,2.723247e+06,0.593804
std,40.133829,1.746533e+07,1.760694e+07,0.257660
min,1882.000000,6.204000e-12,1.707798e-11,0.057906
25%,1916.000000,1.992978e+01,5.126680e+01,0.431571
50%,1951.000000,1.041475e+03,1.794046e+03,0.548365
75%,1986.000000,2.518053e+04,4.556449e+04,0.707641
max,2020.000000,2.357367e+08,2.070466e+08,2.259868


Stock Market Capitalization 

In [113]:
df_stock_market_capitalization_Zimmermann_2022_path = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/Stock Market Capitalization/BBdatasetR1")
df_stock_market_capitalization_Zimmermann_2022 = pd.read_excel(df_stock_market_capitalization_Zimmermann_2022_path/ "Zimmermann 2022.xlsx", sheet_name="Data")
df_stock_market_capitalization_Zimmermann_2022 = df_stock_market_capitalization_Zimmermann_2022[["country", "year", "mcap_gdp"]]

country_replace = {
    "AUS": "Australia",
    "AUT": "Austria",
    "BEL": "Belgium",
    "CAN": "Canada",
    "DNK": "Denmark",
    "FIN": "Finland",
    "FRA": "France",
    "DEU": "Germany",
    "GRC": "Greece",
    "IRL": "Ireland",
    "ITA": "Italy",
    "JPN": "Japan",
    "NLD": "Netherlands",
    "NZL": "New Zealand",
    "NOR": "Norway",
    "PRT": "Portugal",
    "ESP": "Spain",
    "SWE": "Sweden",
    "CHE": "Switzerland",
    "GBR": "United Kingdom",  
    "UK": "United Kingdom",    
    "USA": "United States",
    "US": "United States"      
}

df_stock_market_capitalization_Zimmermann_2022["country"] = df_stock_market_capitalization_Zimmermann_2022["country"].replace(country_replace)
excluded_countries = ["Finland", "Italy", "Japan", "Netherlands"]
df_stock_market_capitalization_Zimmermann_2022 = df_stock_market_capitalization_Zimmermann_2022[~df_stock_market_capitalization_Zimmermann_2022["country"].isin(excluded_countries)]

df_stock_market_capitalization_Zimmermann_2022 = df_stock_market_capitalization_Zimmermann_2022[df_stock_market_capitalization_Zimmermann_2022["country"].isin(Project_Countries)]

df_stock_market_capitalization_Zimmermann_2022["year"] = pd.to_datetime(
    df_stock_market_capitalization_Zimmermann_2022["year"]
)

df_stock_market_capitalization_Zimmermann_2022 = df_stock_market_capitalization_Zimmermann_2022[
    (df_stock_market_capitalization_Zimmermann_2022["year"].dt.year >= 1889) &
    (df_stock_market_capitalization_Zimmermann_2022["year"].dt.year <= 2016)
]

df_stock_market_capitalization_Zimmermann_2022["year"] = pd.to_datetime(
    df_stock_market_capitalization_Zimmermann_2022["year"], errors="coerce"
).dt.year

def interpolate_mcap_for_country_years(df, country, start_year, end_year):
    mask_country = df["country"] == country
    df_country = df.loc[mask_country].sort_values("year").copy()

    # interpolate the stock market capitalization ratio
    df_country["mcap_interp"] = df_country["mcap_gdp"].interpolate(method="linear")

    # now year is int, so this is fine
    mask_gap = (df_country["year"] >= start_year) & (df_country["year"] <= end_year)

    df.loc[mask_country & mask_gap, "mcap_gdp"] = df_country.loc[mask_gap, "mcap_interp"].values
    return df

# Canada (1927–1933)
df_stock_market_capitalization_Zimmermann_2022 = interpolate_mcap_for_country_years(
    df_stock_market_capitalization_Zimmermann_2022, "Canada", 1927, 1933
)

# Germany (1923)
df_stock_market_capitalization_Zimmermann_2022 = interpolate_mcap_for_country_years(
    df_stock_market_capitalization_Zimmermann_2022, "Germany", 1923, 1923
)

# Germany (1945)
df_stock_market_capitalization_Zimmermann_2022 = interpolate_mcap_for_country_years(
    df_stock_market_capitalization_Zimmermann_2022, "Germany", 1945, 1945
)

# France (1917–1919)
df_stock_market_capitalization_Zimmermann_2022 = interpolate_mcap_for_country_years(
    df_stock_market_capitalization_Zimmermann_2022, "France", 1917, 1919
)

# France (1939–1944)
df_stock_market_capitalization_Zimmermann_2022 = interpolate_mcap_for_country_years(
    df_stock_market_capitalization_Zimmermann_2022, "France", 1939, 1944
)

df_stock_market_capitalization_Zimmermann_2022 = interpolate_mcap_for_country_years(
    df_stock_market_capitalization_Zimmermann_2022, "Norway", 1940, 1945
) 

print(df_stock_market_capitalization_Zimmermann_2022[df_stock_market_capitalization_Zimmermann_2022["mcap_gdp"].isna()])






            country  year  mcap_gdp
20        Australia  1889       NaN
21        Australia  1890       NaN
22        Australia  1891       NaN
23        Australia  1892       NaN
24        Australia  1893       NaN
25        Australia  1894       NaN
26        Australia  1895       NaN
27        Australia  1896       NaN
28        Australia  1897       NaN
314          Canada  1889       NaN
315          Canada  1890       NaN
316          Canada  1891       NaN
317          Canada  1892       NaN
318          Canada  1893       NaN
319          Canada  1894       NaN
320          Canada  1895       NaN
321          Canada  1896       NaN
322          Canada  1897       NaN
902           Spain  1889       NaN
903           Spain  1890       NaN
904           Spain  1891       NaN
905           Spain  1892       NaN
906           Spain  1893       NaN
907           Spain  1894       NaN
908           Spain  1895       NaN
909           Spain  1896       NaN
910           Spain  1897   

Bank Loans Data set --> Jorda, Muller and Verner, World Bank, 

In [114]:
df_Bank_Loan_data_path = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/CPI")
df_Bank_Loan_data = pd.read_excel(df_Bank_Loan_data_path / " Jordá et al 2017  - .xlsx", sheet_name='Sheet1')
df_Bank_Loan_GDP_data_path = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/Trade Openness")
df_Bank_Loan_GDP_data = pd.read_excel(df_Bank_Loan_GDP_data_path / "GMD.xlsx", sheet_name='data_final')

country_replace = {
    "AUS": "Australia",
    "AUT": "Austria",
    "BEL": "Belgium",
    "CAN": "Canada",
    "DNK": "Denmark",
    "FIN": "Finland",
    "FRA": "France",
    "DEU": "Germany",
    "GRC": "Greece",
    "IRL": "Ireland",
    "ITA": "Italy",
    "JPN": "Japan",
    "NLD": "Netherlands",
    "NZL": "New Zealand",
    "NOR": "Norway",
    "PRT": "Portugal",
    "ESP": "Spain",
    "SWE": "Sweden",
    "CHE": "Switzerland",
    "GBR": "United Kingdom",  
    "UK": "United Kingdom",    
    "USA": "United States",
    "US": "United States"      
}

df_Bank_Loan_data["country"] = df_Bank_Loan_data["country"].replace(country_replace)
df_Bank_Loan_data = df_Bank_Loan_data[['country', 'year', 'tloans', 'gdp']]
df_Bank_Loan_data = df_Bank_Loan_data[(df_Bank_Loan_data['year'] >= 1882) & (df_Bank_Loan_data['year'] <= 2020)]
df_Bank_Loan_data = df_Bank_Loan_data[df_Bank_Loan_data["country"].isin(Project_Countries)]
df_Bank_Loan_GDP_data = df_Bank_Loan_GDP_data[['countryname', 'year', 'nGDP']]





df_Bank_Loan_GDP_data = df_Bank_Loan_GDP_data[df_Bank_Loan_GDP_data["countryname"].isin(Project_Countries)]
df_Bank_Loan_GDP_data = df_Bank_Loan_GDP_data.rename(columns={'countryname': 'country'})
df_Bank_Loan_GDP_data = df_Bank_Loan_GDP_data[(df_Bank_Loan_GDP_data['year'] >= 1882) & (df_Bank_Loan_GDP_data['year'] <= 2020)]

df_Bank_Loan_data = pd.merge(df_Bank_Loan_data, df_Bank_Loan_GDP_data, on=['country', 'year'], how='left')
df_Bank_Loan_data['bank_loans_gdp'] = df_Bank_Loan_data.apply(
    lambda row: row['tloans'] / row['gdp'] if pd.notna(row['gdp'])
    else (row['tloans'] / row['nGDP'] if pd.notna(row['nGDP']) else None),
    axis=1
)

exclude_countries = ['Spain', 'France', 'Netherlands', 'Portugal']

df_Bank_Loan_data = df_Bank_Loan_data[~df_Bank_Loan_data["country"].isin(exclude_countries)]    
df_Bank_Loan_data.head()

# Following the data cleaning steps illustrated in the paper
df_Bank_Loan_data = df_Bank_Loan_data.sort_values(['country', 'year'])
df_Bank_Loan_data['bank_loans_gdp_interp'] = (df_Bank_Loan_data.groupby('country')['bank_loans_gdp'].transform(lambda s: s.interpolate(method='linear')))
mask_germany = (
(df_Bank_Loan_data['country'] == 'Germany') &
(df_Bank_Loan_data['year'].between(1921, 1923) |df_Bank_Loan_data['year'].between(1941, 1945)))
mask_japan = ((df_Bank_Loan_data['country'] == 'Japan') &(df_Bank_Loan_data['year'] == 1945))
mask_Australia = ((df_Bank_Loan_data['country'] == 'Australia') & (df_Bank_Loan_data['year'].isin([1946, 1947])))
mask = mask_germany | mask_japan | mask_Australia
df_Bank_Loan_data.loc[mask, 'bank_loans_gdp'] = (df_Bank_Loan_data.loc[mask, 'bank_loans_gdp_interp'])
df_Bank_Loan_data = df_Bank_Loan_data.drop(columns='bank_loans_gdp_interp')

# getting missing data for Japan - World Bank 

jp_fix = {
    2018: 147.01,
    2019: 147.83,
    2020: 154.29
}

for yr, val in jp_fix.items():
    mask = (df_Bank_Loan_data['country'] == 'Japan') & (df_Bank_Loan_data['year'] == yr)
    df_Bank_Loan_data.loc[mask, 'bank_loans_gdp'] = val / 100

Merging Everything Together 

In [115]:
df_gdp = df_OECD_GDP.copy()          


df_lp_long = (
    df_labour_productivity
    .melt(id_vars='Year', var_name='country', value_name='labour_productivity')
    .rename(columns={'Year': 'year'})
)
df_cpi = df_CPI_Jorda[['country', 'year', 'cpi']].copy()
df_to = df_TradeOpenness[['country', 'year', 'trade_openness']].copy()
df_ll = df_Liquid_Liabilities[['country', 'year', 'liquid_liabilities']].copy()
df_mcap = df_stock_market_capitalization_Zimmermann_2022[['country', 'year', 'mcap_gdp']].copy()
df_loans = df_Bank_Loan_data[['country', 'year', 'bank_loans_gdp']].copy()

df_Final = (
    df_gdp
      .merge(df_lp_long, on=['country', 'year'], how='left')
      .merge(df_cpi,     on=['country', 'year'], how='left')
      .merge(df_to,      on=['country', 'year'], how='left')
      .merge(df_ll,      on=['country', 'year'], how='left')
      .merge(df_mcap,    on=['country', 'year'], how='left')
      .merge(df_loans,   on=['country', 'year'], how='left')
)

df_Final = df_Final[(df_Final['year'] >= 1882) & (df_Final['year'] <= 2020)]

output_path = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/final_panel_dataset.xlsx")

# Save df_all to Excel
df_Final.to_excel(output_path, index=False)

print("Saved to:", output_path)

Saved to: /Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/final_panel_dataset.xlsx
